In [45]:
#!pip install deap 

In [46]:
import numpy as np
import math
from deap import base, creator, tools
import random
import matplotlib.pyplot as plt
import time
from scipy.optimize import minimize
import json
import pandas as pd


## Tentando colocar dados de entrada

In [47]:
from typing import List, Optional
from pydantic import BaseModel



class ClassIndividual(BaseModel):
    index: int
    var_decicison: List[float]
    Fitness: Optional[float]
    RCE: Optional[str]
    diversidade: float

class SetupInd:
    def __init__(self, params, dadosEntrada=None):

        #! Parametros JSON
        self.params = params
        self.DADOS_ENTRADA = dadosEntrada  # Dados da entrada do usuario

        self.CXPB = params["CROSSOVER"]
        self.MUTPB = params["MUTACAO"]
        self.NGEN = params["NUM_GENERATIONS"]
        self.POP_SIZE = params["POP_SIZE"]
        self.SIZE_INDIVIDUAL = params["IND_SIZE"]
        self.TAXA_GENERATION = params["RCE_REPOPULATION_GENERATIONS"]
        self.CROSSOVER, self.MUTACAO, self.NUM_GENERATIONS, self.POPULATION_SIZE = (
            self.CXPB,
            self.MUTPB,
            self.NGEN,
            self.POP_SIZE,
        )

        self.delta = params["DELTA"]

        #! Parâmetros do algoritmo de Rastrigin
        self.evaluations = 0
        self.num_repopulation = int(self.NUM_GENERATIONS * self.TAXA_GENERATION)
        self.type = params["type"].lower()
        if self.type == "maximize":
            print("Método escolhido: Maximizar")
            creator.create("Fitness", base.Fitness, weights=(1.0,))
        else:
            print("Método escolhido: Minimizar")
            creator.create("Fitness", base.Fitness, weights=(-1.0,))

        self.dataset = {}
        self.beta_constants = {}

        #!Criando individuo pelo deap com seus atributos
        self.toolbox = base.Toolbox()

        # Função de fitness de minimização
        creator.create("FitnessMin", base.Fitness, weights=(-1.0,))

        # parâmetros
        creator.create(
            "Individual", list, fitness=creator.FitnessMin, rce=str, index=int
        )

        # Variavel de decisão
        self.toolbox.register("attribute", random.uniform, -5.12, 5.12)

        # registrando os individuos
        self.toolbox.register(
            "individual",
            tools.initRepeat,
            creator.Individual,
            self.toolbox.attribute,
            n=self.SIZE_INDIVIDUAL,
        )

        #! individuos com dados de entrada
        if self.DADOS_ENTRADA:
            creator.create(
                "FrameworkIndividual",
                list,
                fitness=creator.FitnessMin,
                tipo=self.DADOS_ENTRADA[0],
                rce=str,
            )
            self.toolbox.register(
                "varDecision",
                random.uniform,
                self.DADOS_ENTRADA[2][0],
                self.DADOS_ENTRADA[2][1],
            )
            self.toolbox.register(
                "newIndividual",
                tools.initRepeat,
                creator.FrameworkIndividual,
                self.toolbox.varDecision,
                n=self.DADOS_ENTRADA[1],
            )

            self.toolbox.register(
                "FrameworkPopulation",
                tools.initRepeat,
                list,
                self.toolbox.newIndividual,
            )



        self.toolbox.register("mate", tools.cxTwoPoint)
        self.toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.1)
        self.toolbox.register("select", tools.selTournament, tournsize=3)
        self.toolbox.register("evaluate", self.calculaFitness)
                #! paramentos evolutivos
        self.toolbox.register(
            "population", tools.initRepeat, list, self.toolbox.individual
        )


    def calculaFitness(self, individual):
            if self.type == "minimaze":
                result = minimize(
                    self.rastrigin, x0=np.zeros(self.SIZE_INDIVIDUAL), method="BFGS"
                )
                fitness_value = result.fun
            return fitness_value

    def rastrigin(self, individual):
            self.evaluations += 1
            rastrigin = 10 * self.SIZE_INDIVIDUAL
            for i in range(self.SIZE_INDIVIDUAL):
                rastrigin += individual[i] * individual[i] - 10 * (
                    math.cos(2 * np.pi * individual[i])
                )
            return rastrigin

    def create_population(self):
        new_pop = self.toolbox.population(
            n=self.POP_SIZE
        )
        return new_pop



def load_params(file_path):
    with open(file_path, "r") as file:
        params = json.load(file)
    return params



**Parameters**:
*name* – The name of the class to create.

*base* – A base class from which to inherit.

*attribute* – One or more attributes to add on instantiation of this class, optional.




This above line is exactly the same as defining in the creator module something like the following.

```
create("Foo", list, bar=dict, spam=1)

class Foo(list):
    spam = 1

    def __init__(self):
        self.bar = dict()
```

In [48]:
import easygui

class IMCCalculator:
    def __init__(self,dados):
        self.window_title = "Calculadora de IMC"
        self.nome = dados["nome"]
        self.idade = dados["idade"]
        self.altura = dados["altura"]
        self.peso = dados["peso"]

    def get_input(self):
        msg = "Insira seus dados abaixo:"
        title = self.window_title
        fieldNames = ["nome", "idade", "altura (m)", "peso (kg)"]
        fieldNames_defs = [self.nome, self.idade,self.altura,self.peso]
        fieldValues = easygui.multenterbox(msg, title, fieldNames, fieldNames_defs)

        while 1:
            if fieldValues == None:
                break
            errmsg = ""
            for i in range(len(fieldNames)):
                if fieldValues[i].strip() == "":
                    errmsg = errmsg + ('"%s" is a required field.\n\n' % fieldNames[i])
            if errmsg == "":
                break  # no problems found
            fieldValues = easygui.multenterbox(errmsg, title, fieldNames, fieldValues)
        return fieldValues

    def calculate_imc(self, weight, height):
        if weight <= 0 or height <= 0:
            raise ValueError("Valores negativos não são aceitos.")
        imc = weight / (height * height)
        return imc

    def display_result(self, imc, age, name):
        message = f"Voce com {age} anos, {name}, tem seu IMC é: {imc:.2f}"
        # easygui.msgbox(
        #     message,
        #     self.window_title,
        #     title="Resultado",
        #     ok_button="OK",
        #     image=None,
        #     root=None,
        # )
        print(message)


def main():
    dados = {"nome": "Pedro", "idade": 26, "altura": 1.72, "peso": 70}

    calculator = IMCCalculator(dados)
    try:
        name, age, height, weight = calculator.get_input()
        imc = calculator.calculate_imc(float(weight), float(height))
        calculator.display_result(imc, age, name)
    except ValueError as e:
        easygui.msgbox(str(e), "Erro", title="Erro", ok_button="OK")


#main()

In [49]:
params = load_params(
    "/home/pedrov/Documentos/GitHub/Engenharia-Eletrica-UFF/Iniciação Cientifica - Eng Eletrica UFF/evolution_rce_master/src/db/parameters.json"
)

new_setup = SetupInd(params)

pop = new_setup.create_population()


db_ind: List[ClassIndividual] = []

for i, ind in enumerate(pop):
    print(i, ind)
    print(type(ind))
    print("\n")

    # Create ClassIndividual instance
    individual = ClassIndividual(
        index=i,
        var_decicison=ind,
        diversidade = sum(ind),
        Fitness = (10.0),
        RCE= "rce"
    )

    db_ind.append(individual)

    if (i == 5):
        break

print("tipo ind =",type(db_ind[0].var_decicison))

Método escolhido: Minimizar
0 [-0.42789511127618063, -3.534871880805322, 4.534812821802512, 3.451984883979173, 1.1474175182507675, -2.937024999010622, -0.5415953575002037, -4.682107695663811, 3.042942147355192, -4.6490091771545945]
<class 'deap.creator.Individual'>


1 [4.067967534291026, -2.2294844570465626, -2.180042721576111, 3.076620779941579, 1.7763905086740817, -1.7745939062993399, -2.686973010117081, -1.198753169060526, -3.8813947466956664, -2.194451905263093]
<class 'deap.creator.Individual'>


2 [4.270903279689814, 2.0630187703579885, -4.147649232670889, 2.885860743101701, 0.9143678704090563, 0.49808096033067617, 3.2512585232439912, -0.039737962351726885, 1.6866433235897844, -0.6531645798718948]
<class 'deap.creator.Individual'>


3 [-0.6033251113866163, -0.6129595118172233, 1.2200875659297798, 0.21275266401370452, -4.310822861266716, -2.42017495614239, -3.9440335727915343, -3.7419731115819377, -2.2328905222903903, -3.832813338628938]
<class 'deap.creator.Individual'>


4 [-3.

/home/pedrov/.local/lib/python3.11/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'Fitness' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/pedrov/.local/lib/python3.11/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/pedrov/.local/lib/python3.11/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


## Setup

In [50]:
import numpy as np
import math
from deap import base, creator, tools
import random
import matplotlib.pyplot as plt
import time
from scipy.optimize import minimize
import json
import pandas as pd


class Setup:
    def __init__(self, params, dadosEntrada=None):

        #! Parametros JSON
        self.params = params
        self.DADOS_ENTRADA = dadosEntrada  # Dados da entrada do usuario

        self.CXPB = params["CROSSOVER"]
        self.MUTPB = params["MUTACAO"]
        self.NGEN = params["NUM_GENERATIONS"]
        self.POP_SIZE = params["POP_SIZE"]
        self.SIZE_INDIVIDUAL = params["IND_SIZE"]
        self.TAXA_GENERATION = params["RCE_REPOPULATION_GENERATIONS"]
        self.CROSSOVER, self.MUTACAO, self.NUM_GENERATIONS, self.POPULATION_SIZE = (
            self.CXPB,
            self.MUTPB,
            self.NGEN,
            self.POP_SIZE,
        )

        self.delta = params["DELTA"]
        self.porcentagem = params["PORCENTAGEM"]

        #! Parâmetros do algoritmo de Rastrigin
        self.evaluations = 0
        self.num_repopulation = int(self.NUM_GENERATIONS * self.TAXA_GENERATION)
        self.type = params["type"].lower()
        if self.type == "maximize":
            print("Método escolhido: Maximizar")
            creator.create("Fitness", base.Fitness, weights=(1.0,))
        else:
            print("Método escolhido: Minimizar")
            creator.create("Fitness", base.Fitness, weights=(-1.0,))

        self.dataset = {}
        self.beta_constants = {}

        #!Criando individuo pelo deap com seus atributos
        self.toolbox = base.Toolbox()

        # Função de fitness de minimização
        creator.create("FitnessMin", base.Fitness, weights=(-1.0,))

        # parâmetros
        creator.create(
            "Individual", list, fitness=creator.FitnessMin, rce=str, index=int
        )

        # Variavel de decisão
        self.toolbox.register("attribute", random.uniform, -5.12, 5.12)

        # registrando os individuos
        self.toolbox.register(
            "individual",
            tools.initRepeat,
            creator.Individual,
            self.toolbox.attribute,
            n=self.SIZE_INDIVIDUAL,
        )

        #! individuos com dados de entrada
        if self.DADOS_ENTRADA:
            creator.create(
                "FrameworkIndividual",
                list,
                fitness=creator.FitnessMin,
                tipo=self.DADOS_ENTRADA[0],
                rce=str,
            )
            self.toolbox.register(
                "varDecision",
                random.uniform,
                self.DADOS_ENTRADA[2][0],
                self.DADOS_ENTRADA[2][1],
            )
            self.toolbox.register(
                "newIndividual",
                tools.initRepeat,
                creator.FrameworkIndividual,
                self.toolbox.varDecision,
                n=self.DADOS_ENTRADA[1],
            )

            self.toolbox.register(
                "FrameworkPopulation",
                tools.initRepeat,
                list,
                self.toolbox.newIndividual,
            )

        #! paramentos evolutivos
        self.toolbox.register(
            "population", tools.initRepeat, list, self.toolbox.individual
        )

        self.toolbox.register("mate", tools.cxTwoPoint)
        self.toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.1)
        self.toolbox.register("select", tools.selTournament, tournsize=3)
        self.toolbox.register("evaluate", self.evaluate_fitness)

    def evaluate_fitness(self, individual):
        if self.type == "minimaze":
            result = minimize(
                self.rastrigin, x0=np.zeros(self.SIZE_INDIVIDUAL), method="BFGS"
            )
            fitness_value = result.fun
        return fitness_value

    def gerarDataset(self, excel):
        df = pd.read_excel(excel)
        print(df.columns)
        self.dataset = {
            "CXPB": self.CROSSOVER,
            "TAXA_MUTACAO": self.MUTACAO,
            "NUM_GEN": self.NUM_GENERATIONS,
            "POP_SIZE": self.POPULATION_SIZE,
            "IND_SIZE": self.SIZE_INDIVIDUAL,
            "evaluations": self.evaluations,
            "NUM_REPOPULATION": self.num_repopulation,
        }

    def rastrigin(self, individual):
        self.evaluations += 1
        rastrigin = 10 * self.SIZE_INDIVIDUAL
        for i in range(self.SIZE_INDIVIDUAL):
            rastrigin += individual[i] * individual[i] - 10 * (
                math.cos(2 * np.pi * individual[i])
            )
        return rastrigin

    def rastrigin_decisionVariables(self, individual, decision_variables):
        self.evaluations += 1
        rastrigin = 10 * len(decision_variables)
        for i in range(len(decision_variables)):
            rastrigin += individual[i] * individual[i] - 10 * (
                math.cos(2 * np.pi * individual[i])
            )
        return rastrigin

    def rosenbrock(self, x):
        return np.sum(100 * (x[1:] - x[:-1] ** 2) ** 2 + (1 - x[:-1]) ** 2)

    def globalSolutions(self):
        n_dimensions = 2

        try:
            rastrigin_result = minimize(
                self.rastrigin, x0=np.zeros(n_dimensions), method="BFGS"
            )
            rastrigin_minimum = rastrigin_result.fun
            rastrigin_solution = rastrigin_result.x

            rosenbrock_result = minimize(
                self.rosenbrock, x0=np.zeros(n_dimensions), method="BFGS"
            )
            rosenbrock_minimum = rosenbrock_result.fun
            rosenbrock_solution = rosenbrock_result.x

        except Exception as e:
            print("Erro ao tentar encontrar o ótimo global das funções: ", e)

        print("\n\nÓtimo global da função Rastrigin: ", rastrigin_minimum)
        print("Solução: ", rastrigin_solution)
        print()
        print("Ótimo global da função Rosenbrock: ", rosenbrock_minimum)
        print("Solução: ", rosenbrock_solution)

## COLOCANDO PARAMETROS

-----------------------
def  Critério_1
Aptidão do indivíduo é menor ou igual que (1 + parametro_perc) * aptidão do melhor indivíduo

Para todos os indivíduos da pop atual
Se individuo atende ao critério 1
     inserir HOF
Se lista vazia
          inserir individuo
     Senão
          Se individuo é diferente de todos da lista
                inserir individuo
-----------------------
def compararIndividuosSelecionados():
Para todos indivíduos da lista
       def Calcular Diff - diferença entre as variáveis de indivíduo e individuo lista
       Se Diff não é maior que parâmetro_diff em mais de parâmetro_var -> numero de variaveis diferentes (min 2)
             Retorna falso
Fim Para
Retorna verdadeiro



In [51]:
from deap import base, creator

def load_params(file_path):
    with open(file_path, "r") as file:
        params = json.load(file)
    return params

params = load_params(
    r"/home/pedrov/Documentos/GitHub/Engenharia-Eletrica-UFF/Iniciação Cientifica - Eng Eletrica UFF/evolution_rce_master/src/db/parameters.json"
)

# Criando uma instância da classe Setup
setup = Setup(params)


#! Carregar os dados
turbina = pd.read_csv(
    r"/home/pedrov/Documentos/GitHub/Engenharia-Eletrica-UFF/Iniciação Cientifica - Eng Eletrica UFF/evolution_rce_master/src/assets/datasets/T1.csv"
)
turbina.columns = ['Data/hora', 'ActivePower(kW)', 'WindSpeed(m/s)', 'Curva_Teórica(KWh)',"Direção do Vento"]
display(turbina.head())

X = turbina[['WindSpeed(m/s)']].values
y = turbina[['ActivePower(kW)']].values
data_hora = turbina[['Data/hora']].values
curva = turbina[['Curva_Teórica(KWh)']].values
vento = turbina[['Direção do Vento']].values

Método escolhido: Minimizar


,Data/hora,ActivePower(kW),WindSpeed(m/s),Curva_Teórica(KWh),Direção do Vento
0,01 01 2018 00:00,380.047791,5.311336,416.328908,259.994904
1,01 01 2018 00:10,453.769196,5.672167,519.917511,268.641113
2,01 01 2018 00:20,306.376587,5.216037,390.900016,272.564789
3,01 01 2018 00:30,419.645905,5.659674,516.127569,271.258087
4,01 01 2018 00:40,380.650696,5.577941,491.702972,265.674286


## Algoritimo Evolutivo

In [52]:
import numpy as np
import math
from deap import base, creator, tools
import random
import matplotlib.pyplot as plt
import time
from scipy.optimize import minimize
import json
import pandas as pd
import numpy as np
import math
from deap import base, creator, tools
import random
import matplotlib.pyplot as plt
import time
from scipy.optimize import minimize
import json
import pandas as pd


class AlgoritimoEvolutivoRCE:
    def __init__(self, setup):
        self.setup = setup
        self.stats = tools.Statistics(key=lambda ind: ind.fitness.values)
        self.stats.register("avg", np.mean)
        self.stats.register("std", np.std)
        self.stats.register("min", np.min)
        self.stats.register("max", np.max)

        self.logbook = tools.Logbook()
        self.hof = tools.HallOfFame(1)
        self.POPULATION = self.setup.toolbox.population(n=self.setup.POP_SIZE)
        self.hof.update(self.POPULATION)

        if self.setup.DADOS_ENTRADA:
            self.POP_OPTIMIZATION = self.setup.toolbox.population(n=self.setup.POP_SIZE)

        self.pop_RCE = []
        self.best_solutions_array = []
        self.best_individual_array = []
        self.allIndividualValuesArray = []
        self.data = {}
        self.repopulation_counter = 0
        self.allFitnessValues = {}
        self.validateCounter = 0
        self.CONJUNTO_ELITE_RCE = set()

    def registrarDados(self, generation):

        # Registrar estatísticas e melhores soluções
        for ind in self.POPULATION:
            avg_fitness_per_generation = np.mean(ind.fitness.values)
            std_deviation = np.std(ind.fitness.values)

        #! PEgandos os dados e colocando no df
        self.data = {
            "Generations": generation + 2,
            "Variaveis de Decisão": self.hof[0],
            "Evaluations": self.setup.evaluations,
            "Ind Valido": self.hof[0].fitness.valid,
            "Best Fitness": self.hof[0].fitness.values,
            "Media": avg_fitness_per_generation,
            "Desvio Padrao": std_deviation,
        }

        self.best_individual_array.append(self.data)

        self.visualizarPopAtual(generation, [avg_fitness_per_generation, std_deviation])

    def checkClonesInPop(self, ind, new_pop):
        is_clone = False
        for other_ind in new_pop:
            if (
                ind == other_ind
                and sum(ind) == sum(other_ind)
                and ind.index != other_ind.index
            ):
                is_clone = True
                break
        return is_clone

    def generateInfoIndividual(self, new_pop, generation):
        ind_array = []

        for i, ind in enumerate(new_pop):
            # print(f"Index[{ind.index}] - ind_variables {ind} \n Fitness = {ind.fitness.values} ")

            ind.index = i

            ind_info = {
                "Generations": generation,
                "index": ind.index,
                "Variaveis de Decisão": ind,
                "Fitness": ind.fitness.values[0],
                "RCE": ind.rce,
                "Diversidade": np.sum(ind),
            }

            # Adicionar a informação de clone ao dicionário
            # is_clone = self.checkClonesInPop(ind, new_pop)
            # ind_info["CLONE"] = "SIM" if is_clone else "NAO"

            ind_array.append(ind_info)

        return ind_array

    def show_ind_df(self, array, text):
        df = pd.DataFrame(array)
        print(text)
        display(df.head(40))

        # contar quantos SIM na coluna CLONE se a coluna RCE for SIM
        # display(df[df["RCE"] != ""]["CLONE"].value_counts())

    def criterio1(self, new_pop, porcentagem, k=30):
        """Seleciona os candidatos ao conjunto elite com base nas diferenças percentuais de aptidão."""
        self.cout(f"CRITÉRIO 1 RCE - Selecionando candidatos ao conjunto elite")
        elite_individuals = []

        # Ordenar a população em ordem crescente de aptidão e selecionar os k primeiros indivíduos
        sorted_population = sorted(self.POPULATION, key=lambda x: x.fitness.values[0])

        # Obter o melhor indivíduo (HOF) da população
        best_ind = new_pop[0]
        best_fitness = best_ind.fitness.values[0]
        max_difference = (1 + porcentagem) * best_fitness

        # Selecionar com as menores diferenças percentuais
        for ind in sorted_population:
            if ind.fitness.values[0] <= max_difference:
                elite_individuals.append(ind)
            else:
                break  # Parar a seleção quando a diferença percentual for maior que o limite

        # Colocando na pop aleatória
        print(
            f"Calculando percentual de {porcentagem*100}% com base no melhor fitness = {best_fitness} e pegando os {len(elite_individuals)} melhores.\n Porcentagem de {best_fitness} = {max_difference} "
        )
        for i, ind in enumerate(elite_individuals):
            new_pop[i] = self.setup.toolbox.clone(ind)
            new_pop[i].rce = "SIM_1"

        return elite_individuals

    def criterio2_alternative(self, ind_selecionados, delta=6):
        """Comparar as variáveis de decisão de cada indivíduo e verificar se existem diferenças superiores a 'delta'."""
        self.cout(
            f"CRITÉRIO 2 - Comparar as variáveis de decisão de cada indivíduo e verificar valores superiores a 'delta' = {delta}."
        )
        self.CONJUNTO_ELITE_RCE.clear()

        for i in range(len(ind_selecionados)):

            #    todo   Calcular Diff - diferença entre as variáveis de indivíduo e individuo lista

            diff = np.array(ind_selecionados[i]) - np.array(ind_selecionados[0])

            # todo retornar true ou false caso ind seja diferente para colocar no array correto
            if sum(diff) > delta:  # ind diferente
                if (ind_selecionados[i] not in self.pop_RCE) and (
                    ind_selecionados[i] not in ind_selecionados[0]
                ):
                    self.pop_RCE.append(self.POP_OPTIMIZATION[i])
                    self.CONJUNTO_ELITE_RCE.add(tuple(self.POP_OPTIMIZATION[i]))
                    ##print("Delta= ", sum(diff))

        if not self.pop_RCE:
            print("Nenhum indivíduo atende aos critérios. :( ")

        print("Tamanho Elite = ", len(self.pop_RCE))
        print("Tamanho Elite = ", len(self.CONJUNTO_ELITE_RCE))

        return self.pop_RCE

    def newCriterio(self, population, delta):
        self.cout(f"New - CRITÉRIO 2 RCE ")
        self.CONJUNTO_ELITE_RCE.clear()
        self.pop_RCE = []

        def criterio1_reduzido(population ):
            best_ind = self.elitismoSimples(population)[0]
            best_fitness = best_ind.fitness.values[0]
            max_difference = (1 + self.setup.porcentagem) * best_fitness
            print(
                f"Fitness calculado com {self.setup.porcentagem}% = {round(max_difference,3)}"
            )
            return max_difference, best_ind

        max_difference, best_ind = criterio1_reduzido(population)
        self.pop_RCE.append(best_ind)

        # todo colocar hof no rce e ajustar parâmetros sim e nao
        def calculaDiff(ind, lista):
            count = 0
            print(ind)

            for i in range(len(lista)):
                diff = abs(np.array(ind) - np.array(lista[i]))
                array = list(diff)

                if sum(diff) > 0.0:
                    print("array selecionado: ", list(diff))
                    print("\n Soma Diff = ", sum(diff))
                    print("contador = ", count)

                    for j in range(len(array)):
                        print("debug", array[j])

                        if array[j] >= self.setup.delta:
                            count += 1

                        if j == len(lista) - 1:
                            print("Não é diferente ")
                            return False
                            break

                    if count >= self.setup.delta:
                        print("Ind Diferente RCE")
                        return True
                    else:
                        return False

                else:
                    return False

        for ind in population:
            # criterio 1
            if ind.fitness.values[0] <= max_difference:
                # criterio 2
                diff = calculaDiff(ind, self.pop_RCE) # delta como valor limite
                if diff:
                    if ind not in self.pop_RCE:
                        self.pop_RCE.append(ind)
                        self.CONJUNTO_ELITE_RCE.add(tuple(ind))

        if len(self.pop_RCE) == 1:
            print("Nenhum indivíduo atende aos critérios. :( ")

        print("Tamanho Elite = ", len(self.pop_RCE))
        print("Tamanho Elite = ", len(self.CONJUNTO_ELITE_RCE))

        return self.pop_RCE

    def aplicar_RCE(self, generation, current_population):

        #! a - Cria uma pop aleatória (eliminando a pop aleatória criada na execução anterior do RCE)
        new_pop = self.setup.toolbox.population(
            n=self.setup.POP_SIZE
        )  # retorna uma pop com lista de individuos de var de decisão

        # Avaliar o fitness da população atual
        self.avaliarFitnessIndividuos(current_population)
        self.calculateFitnessGeneration(current_population)

        #! b - Coloca o elite hof da pop anterior  no topo (0)
        pop = self.elitismoSimples(current_population)
        print(
            f"Elitismo HOF Index[{pop[0].index}] {pop[0]} \n Fitness = {pop[0].fitness.values} | Diversidade = {sum(pop[0])}"
        )  # pop[0] é o melhor individuo HOF
        new_pop[0] = self.setup.toolbox.clone(pop[0])

        #! critério 1 e obtém os N melhores com 30% do valor do melhor fitness
        # ind_selecionados = self.criterio1(self.POPULATION, 0.30, k=30)

        #! Critério 2 usando este array e vai colocando os indivíduos selecionados pelo critério 2 na pop aleatória (passo a)
        # ind_diferentes_var = self.criterio2_alternative(ind_selecionados, delta=7)
        ind_diferentes_var = self.newCriterio(
            current_population, delta=self.setup.delta
        )

        # cOLOCANDO ATRUBUTOS
        for i, ind in enumerate(ind_diferentes_var, start=0):
            new_pop[0].rce = "HOF"
            if i > 0:
                new_pop[i] = self.setup.toolbox.clone(ind)
                new_pop[i].rce = "SIM"

        #! Criterio 3 retorna pop aleatória modificada (com hof + rce + Aleatorio)
        self.calculateFitnessGeneration(new_pop)
        self.cout(f"CRITERIO 3 - População aleatória modificada [HOF,RCE,Aleatorio] ")
        conjunto_elite = self.generateInfoIndividual(new_pop, generation)
        self.show_ind_df(conjunto_elite, "Individuos da nova população aleatória")
        return new_pop

    def elitismoSimples(self, pop):
        self.hof.update(pop)
        pop[0] = self.setup.toolbox.clone(self.hof[0])
        return pop

    def criterio2(self, elite_individuals, delta):
        """Comparar as variáveis de decisão de cada indivíduo e verificar se existem diferenças superiores a 'delta'."""
        self.cout(
            f"CRITÉRIO 2 - Comparar as variáveis de decisão de cada indivíduo e verificar se existem diferenças superiores a 'delta' = {delta}."
        )
        self.pop_RCE = []
        self.CONJUNTO_ELITE_RCE.clear()

        for i in range(len(elite_individuals)):
            current_individual = elite_individuals[i]
            is_diferente = False

            for j in range(i + 1, len(elite_individuals)):
                other_individual = elite_individuals[j]
                diff_counter = 0

                for var_index in range(len(current_individual)):
                    current_var = current_individual[var_index]
                    other_var = other_individual[var_index]

                    if abs(current_var - other_var) > delta:
                        # print(abs(current_var - other_var))
                        diff_counter += 1

                if diff_counter >= 1:
                    is_diferente = True

            if is_diferente:
                # print(f"Indivíduo do tipo {type(current_individual)} VAR({current_individual})\n diferente! adicionado à nova população.")
                self.pop_RCE.append(current_individual)
                self.CONJUNTO_ELITE_RCE.add(tuple(current_individual))

        if not self.pop_RCE:
            print("Nenhum indivíduo atende aos critérios. :( ")

        print("Tamanho Elite = ", len(self.pop_RCE))
        return self.pop_RCE

    def avaliarFitnessIndividuos(self, pop):
        """Avaliar o fitness dos indivíduos da população atual."""
        fitnesses = map(self.setup.toolbox.evaluate, pop)
        for ind, fit in zip(pop, fitnesses):
            if ind.fitness.values:
                ind.fitness.values = [fit]

    def calculateFitnessGeneration(self, new_pop):
        # Calculando o fitness para geração
        for ind in new_pop:
            if not ind.fitness.valid:
                fitness_value = self.setup.toolbox.evaluate(ind)
                ind.fitness.values = (fitness_value,)

    def checkDecisionVariablesAndFitnessFunction(
        self, decision_variables, fitness_function
    ):

        # Verificar se as variáveis de decisão e a função de fitness foram fornecidas
        if decision_variables is None and fitness_function is None:
            # Gerar variáveis de decisão aleatórias para os indivíduos
            decision_variables = [
                random.random() for _ in range(self.setup.SIZE_INDIVIDUAL)
            ]

            # Definir a função de fitness padrão como a função Rastrigin
            fitness_function = self.setup.rastrigin_decisionVariables

        if decision_variables is None or fitness_function is None:
            if not hasattr(self, "decision_variables") or not hasattr(
                self, "fitness_function"
            ):
                raise ValueError(
                    "Variáveis de decisão e função de fitness não definidas. Use set_decision_variables_and_fitness_function primeiro."
                )
        else:
            self.decision_variables = decision_variables
            self.fitness_function = fitness_function

            # Definir a função de fitness com base na função fornecida
            def fitness_func(individual):
                return self.fitness_function(individual, self.decision_variables)

            # Registrar a função de fitness no toolbox
            self.setup.toolbox.register("evaluate", fitness_func)

    #! Main LOOP
    def run(self, RCE=False, decision_variables=None, fitness_function=None, num_pop=0):

        if self.setup.DADOS_ENTRADA:
            population = [self.POPULATION, self.POP_OPTIMIZATION]
        else:
            population = [self.POPULATION]

        # Avaliar o fitness da população inicial
        self.avaliarFitnessIndividuos(population[num_pop])

        # Selecionando as variaveis de decisao e afuncao objeti
        self.checkDecisionVariablesAndFitnessFunction(
            decision_variables, fitness_function
        )

        #! Loop principal através das gerações
        for current_generation in range(self.setup.NGEN):

            # Selecionar os indivíduos para reprodução
            offspring = self.setup.toolbox.select(
                population[num_pop], k=len(population[num_pop])
            )

            # Clone the selected individuals
            offspring = [self.setup.toolbox.clone(ind) for ind in offspring]

            # Aplicar crossover
            for child1, child2 in zip(offspring[::2], offspring[1::2]):
                if random.random() < self.setup.CXPB:
                    self.setup.toolbox.mate(child1, child2)
                    del child1.fitness.values
                    del child2.fitness.values

            # Aplicar mutação
            for mutant in offspring:
                if random.random() < self.setup.MUTPB:
                    self.setup.toolbox.mutate(mutant)
                    del mutant.fitness.values

            #  Avaliar o fitness dos novos indivíduos
            invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
            fitnesses = map(self.setup.toolbox.evaluate, invalid_ind)
            for ind, fit in zip(invalid_ind, fitnesses):
                ind.fitness.values = [fit]

            #! Aplicar RCE
            if RCE and ((current_generation + 1) % self.setup.num_repopulation == 0):
                print("")
                self.cout(
                    f"RCE being applied! - Generation = {current_generation + 1} ",
                )
                #! f - copia pop aleatória modificada retornada para pop atual
                new_population = self.aplicar_RCE(
                    current_generation + 1, population[num_pop]
                )
                # print("\nPopulação gerada pelo RCE\n", population[num_pop])
                population[num_pop][:] = new_population
            else:
                population[num_pop][:] = offspring

            # Registrar estatísticas no logbook
            self.elitismoSimples(population[num_pop])
            self.registrarDados(current_generation)
            record = self.stats.compile(population[num_pop])
            self.logbook.record(gen=current_generation, **record)

        # Retornar população final, logbook e elite
        return population[num_pop], self.logbook, self.hof[0]

    def visualizarPopAtual(self, geracaoAtual, stats):

        for i in range(len(self.POPULATION)):
            datasetIndividuals = {
                "Generations": geracaoAtual + 1,
                "index": i,
                "Variaveis de Decisão": self.POPULATION[i],
                "Fitness": self.POPULATION[i].fitness.values,
                "Media": stats[0],
                "Desvio Padrao": stats[1],
                "RCE": " - ",
            }
            self.allIndividualValuesArray.append(datasetIndividuals)

    def cout(self, msg):
        print(
            "\n=========================================================================================================="
        )
        print("\t", msg)
        print(
            "==========================================================================================================\n"
        )

## Exemplo

In [53]:
import time
import json
import pandas as pd
import numpy as np
from IPython.display import display
import matplotlib.pyplot as plt
import math


# Exemplo dados tubina eolica

X = turbina[['WindSpeed(m/s)']].values
y = turbina[['ActivePower(kW)']].values
data_hora = turbina[['Data/hora']].values
curva = turbina[['Curva_Teórica(KWh)']].values
vento = turbina[['Direção do Vento']].values

print(X.shape,y.shape)


def rastrigin_decisionVariables( individual, decision_variables):
        rastrigin = 10 * len(decision_variables)
        for i in range(len(decision_variables)):
            rastrigin += individual[i] * individual[i] - 10 * (
                math.cos(2 * np.pi * individual[i])
            )
        return rastrigin

turbina = {
    "num_var":5,
    "tipo": int,
    "fitness": rastrigin_decisionVariables
}

#turbina["var"] = [x,y,data_hora, curva, vento]
print(turbina)




(50530, 1) (50530, 1)
{'num_var': 5, 'tipo': <class 'int'>, 'fitness': <function rastrigin_decisionVariables at 0x763ec49e1440>}


In [54]:
## problema da mochila - combinatria do tipo binario

# SA -> https://github.com/brunodeathayde/SA_TSP/blob/main/main_SA.py

#evolution
prob_dict = {
    "tipo": "binario",
    "num_var": 8,
    "pop": 10,
    "fitness": 0,
}

#problema
multa = 1000
capacidade_mochila = 15
quantidade_itens = [1,8,3,12,7,5,6,3]
lucro = [2,5,3,10,8,1,3,4]
fitness = lambda x: (capacidade_mochila - sum(quantidade_itens[x])) * multa * sum(lucro)

#print(fitness(2))

In [55]:
def pesquisaLinear(array,num,delta = 2):
    pos = 0
    count = 0

    while ((pos < len(array)) and (array[pos]) != num):
        pos = pos + 1

    # pegando quantos valores nao sao nulos
    for value in array:
        if value != 0.0:
          count += 1
    print("Valores não nulos = ",count)

    
    # valores a presquisar
    try:
        if (array[pos] == num):
            return True
    except:
        print("nao encontrado")

    # se o contador for maior que delta
    if (count >= delta):
        print("Array diferente")
        return True

    else:
        return False

pesquisaLinear(quantidade_itens,4)

Valores não nulos =  8
nao encontrado
Array diferente


True

## Main

In [56]:
if __name__ == "__main__":
    tempo1 = time.time()

    # Setup
    params = load_params(
        r"/home/pedrov/Documentos/GitHub/Engenharia-Eletrica-UFF/Iniciação Cientifica - Eng Eletrica UFF/evolution_rce_master/src/db/parameters.json"
    )
    setup = Setup(params)
    alg = AlgoritimoEvolutivoRCE(setup)
    # data_visual = DataExploration()
    # old_alg = AlgEvolution(setup)

    pop_with_repopulation, logbook_with_repopulation, best_variables = alg.run(
        RCE=True,
        #fitness_function=rastrigin_decisionVariables,
        #decision_variables=(X, y,data_hora,curva,vento),
    )

    print("\n\nEvolução concluída  - 100%")

    # Salvar resultados
    alg.cout("---> RESULTADOS OBTIDOS")
    # data_visual.displayDataFrameInfo(alg.allIndividualValuesArray, "Todos Individuos Gerados")
    # data_visual.show_conjuntoElite(alg.pop_RCE)

    # Visualização dos resultados
    alg.cout("VISUALIZANDO OS RESULTADOS")
    # data_visual.show_rastrigin_benchmark(logbook_with_repopulation,best_variables)
    # data_visual.visualize(
    #   logbook_with_repopulation, pop_with_repopulation, repopulation=True
    # )
    # data_visual.statistics_per_generation_df(logbook_with_repopulation)

    tempo2 = time.time()
    print(f"Total execution time: {round(tempo2-tempo1,2)} seconds")

Método escolhido: Minimizar


/home/pedrov/.local/lib/python3.11/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'Fitness' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/pedrov/.local/lib/python3.11/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/pedrov/.local/lib/python3.11/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "




	 RCE being applied! - Generation = 20 

Elitismo HOF Index[0] [-0.06916398187872907, 0.982904827242165, -1.0840061141429889, 1.0280647088277748, -0.04165832674011728, -0.053352305907784725, -1.0881501053488387, 0.002442745970444804, 1.9998032218859434, 1.0438077648339705] 
 Fitness = (14.753292085184771,) | Diversidade = 2.72069243474184

	 CRITÉRIO 2 RCE 

Fitness calculado com 0.4 = 21
[-0.06916398187872907, 0.982904827242165, -1.0840061141429889, 1.0280647088277748, -0.04165832674011728, -0.053352305907784725, -1.0881501053488387, 0.002442745970444804, 1.9998032218859434, 1.0438077648339705]
[-0.06916398187872907, 0.982904827242165, -1.0840061141429889, 1.0280647088277748, -0.04165832674011728, -0.053352305907784725, -1.0881501053488387, 0.002442745970444804, 2.107502703784057, 1.0438077648339705]
array selecionado:  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.10769948189811362, 0.0]

 Soma Diff =  0.10769948189811362
contador =  0
debug 0.0
Não é diferente 
[-0.06916398187872907,

,Generations,index,Variaveis de Decisão,Fitness,RCE,Diversidade
0,20,0,"[-0.06916398187872907, 0.982904827242165, -1.0...",14.753292,HOF,2.720692
1,20,1,"[3.0528289245048112, 2.779947272434897, -4.559...",150.663379,,-3.659058
2,20,2,"[-1.852064042957693, -3.1909355148571286, 2.76...",206.614371,,3.581479
3,20,3,"[-4.030281301517508, -1.6805967111392421, 3.65...",216.163438,,10.647553
4,20,4,"[-4.382968164332131, -3.5953987282266815, -4.3...",255.753462,,5.105699
5,20,5,"[-0.0589257339557232, -4.611849049135828, 4.46...",203.056852,,-0.777459
6,20,6,"[-3.4821684060054316, -2.6465724285883288, -4....",228.253161,,-9.250642
7,20,7,"[-1.2080931282114502, 4.309902494513891, 0.447...",234.695626,,23.176212
8,20,8,"[3.111593799018398, 1.6490503668169518, -1.987...",132.280035,,-2.383247
9,20,9,"[0.9061734328919382, 3.0565760932396513, 3.653...",126.470399,,14.054361




	 RCE being applied! - Generation = 40 

Elitismo HOF Index[0] [-0.06916398187872907, -0.01280345999409871, -0.01643659976558265, 1.0280647088277748, 0.03406507187057335, -0.053352305907784725, -1.0881501053488387, 0.002442745970444804, 1.9998032218859434, 1.0438077648339705] 
 Fitness = (11.166590777226123,) | Diversidade = 2.868277060493673

	 CRITÉRIO 2 RCE 

Fitness calculado com 0.4 = 16
[-0.06916398187872907, -0.01280345999409871, -0.01643659976558265, 1.0280647088277748, 0.03406507187057335, -0.053352305907784725, -1.0881501053488387, 0.002442745970444804, 1.9998032218859434, 1.0438077648339705]
[-0.06916398187872907, -0.01280345999409871, -0.01643659976558265, 1.0280647088277748, -0.04165832674011728, -0.053352305907784725, -1.0881501053488387, 0.002442745970444804, 1.9998032218859434, 1.0438077648339705]
array selecionado:  [0.0, 0.0, 0.0, 0.0, 0.07572339861069063, 0.0, 0.0, 0.0, 0.0, 0.0]

 Soma Diff =  0.07572339861069063
contador =  0
debug 0.0
Não é diferente 
[-0.069163

...



	 RCE being applied! - Generation = 80 

Elitismo HOF Index[0] [-0.06916398187872907, -0.01280345999409871, 0.006143806775821048, 1.0280647088277748, 0.03406507187057335, -0.053352305907784725, 0.017857922644826663, 0.002442745970444804, 1.0336446769684726, 1.0438077648339705] 
 Fitness = (5.796458075250227,) | Diversidade = 3.0307069501112713

	 CRITÉRIO 2 RCE 

Fitness calculado com 0.4 = 8
[-0.06916398187872907, -0.01280345999409871, 0.006143806775821048, 1.0280647088277748, 0.03406507187057335, -0.053352305907784725, 0.017857922644826663, 0.002442745970444804, 1.0336446769684726, 1.0438077648339705]
[-0.06916398187872907, -0.01280345999409871, 0.006143806775821048, 1.0280647088277748, 0.03406507187057335, -0.053352305907784725, 0.017857922644826663, 0.002442745970444804, 1.0336446769684726, 1.0438077648339705]
Nenhum indivíduo atende aos critérios. :( 
Tamanho Elite =  1
Tamanho Elite =  0

	 CRITERIO 3 - População aleatória modificada [HOF,RCE,Aleatorio] 

Individuos da nova pop

,Generations,index,Variaveis de Decisão,Fitness,RCE,Diversidade
0,80,0,"[-0.06916398187872907, -0.01280345999409871, 0...",5.796458,HOF,3.030707
1,80,1,"[-3.9871477775084694, -2.9997624106825187, -3....",169.414815,,-6.555603
2,80,2,"[-3.6785502673541544, 2.2281534742413003, 1.44...",200.880319,,0.923926
3,80,3,"[-1.62124644680438, -4.259530187748011, 1.2744...",182.997792,,-13.084021
4,80,4,"[-0.9194532650913061, 5.024440420811689, -1.03...",122.722455,,2.586442
5,80,5,"[1.3144150019783751, 2.803527981010511, 5.0950...",172.543603,,15.458476
6,80,6,"[-4.955614927779983, 2.6914500303064184, -1.51...",193.372617,,-9.864060
7,80,7,"[-2.961461489443285, 4.45116197018105, 4.15555...",207.945252,,4.267093
8,80,8,"[1.6702764383725652, -1.7620329008378453, -4.4...",169.544931,,2.040200
9,80,9,"[-1.121415985309611, -1.4814910582555854, 4.66...",198.292500,,-4.935511




	 RCE being applied! - Generation = 100 

Elitismo HOF Index[0] [-0.06916398187872907, -0.01280345999409871, 0.006143806775821048, 1.0280647088277748, 0.03406507187057335, -0.053352305907784725, -0.008096008038162017, 0.002442745970444804, -0.07736662947161932, 1.0438077648339705] 
 Fitness = (5.619636601367146,) | Diversidade = 1.8937417129881906

	 CRITÉRIO 2 RCE 

Fitness calculado com 0.4 = 8
[-0.06916398187872907, -0.01280345999409871, 0.006143806775821048, 1.0280647088277748, 0.03406507187057335, -0.053352305907784725, -0.008096008038162017, 0.002442745970444804, -0.07736662947161932, 1.0438077648339705]
[-0.06916398187872907, -0.01280345999409871, 0.006143806775821048, 1.0280647088277748, 0.03406507187057335, -0.053352305907784725, 0.017857922644826663, 0.002442745970444804, -0.07736662947161932, 1.0438077648339705]
array selecionado:  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.02595393068298868, 0.0, 0.0, 0.0]

 Soma Diff =  0.02595393068298868
contador =  0
debug 0.0
Não é diferente 


,Generations,index,Variaveis de Decisão,Fitness,RCE,Diversidade
0,100,0,"[-0.06916398187872907, -0.01280345999409871, 0...",5.619637,HOF,1.893742
1,100,1,"[0.4880638395402874, -2.647235602929761, -2.77...",170.136658,,-5.797043
2,100,2,"[-1.2523173894530353, 2.5182966992784053, -4.1...",188.856459,,0.449742
3,100,3,"[-1.9136469076185074, -1.2352686750621023, 1.7...",228.588907,,-12.629886
4,100,4,"[4.3693174834435995, -4.344199558778516, 3.498...",253.055641,,2.473344
5,100,5,"[4.749138262472459, -3.312665380085977, 2.1914...",183.525262,,7.193504
6,100,6,"[0.007379652855966867, 4.764550665520679, -2.7...",158.306045,,3.712673
7,100,7,"[-4.220916563481459, -3.5271487962244965, 0.55...",204.726472,,-3.079476
8,100,8,"[-0.16860092454667974, -1.787897578464405, 2.8...",174.030550,,-8.208713
9,100,9,"[4.478607233471501, -4.64986944857022, -1.7238...",239.554180,,7.910861




Evolução concluída  - 100%

	 ---> RESULTADOS OBTIDOS


	 VISUALIZANDO OS RESULTADOS

Total execution time: 1.64 seconds
